# Importing required libraries

In [ ]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import os
import math
import sys
from datetime import datetime as dt
from geopy.geocoders import Nominatim
#from folium import Chloropleth
#from folium import Circle
#from folium import marker
from folium.plugins import HeatMap, MarkerCluster
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Path where all the crime report files are availab;e
folder_path = r'/content/drive/MyDrive/Boston_crime'

# Change the current working directory as the above folder path
os.chdir(folder_path)

files=os.listdir()

files

['crime_report_2015.csv',
 'crime_report_2016.csv',
 'crime_report_2017.csv',
 'crime_report_2018.csv',
 'crime_report_2019.csv',
 'crime_report_2020.csv',
 'crime_report_2021.csv',
 'crime_report_2022.csv',
 'crime-incident-reports-2015.csv',
 'crime-incident-reports-2016.csv',
 'crime-incident-reports-2017.csv',
 'crime-incident-reports-2018.csv',
 'crime-incident-reports-2019.csv',
 'crime-incident-reports-2020.csv',
 'crime-incident-reports-2022.csv',
 'rmsoffensecodes.xlsx',
 'crime_report_2023.csv',
 'crime_reports_cleaned.csv',
 'Count_Violent_vs_nonviolent.png',
 'crime_y2018.html',
 'crime_y2019.html',
 'crime_y2020.html',
 'crime_y2021.html',
 'crime_2020_clustermap.html',
 'violent_crime_2021_heatmap.html',
 'Crime_Categories.png']

In [ ]:
# Filtering for the files that start with 'crime_report_'
csv_files = list(filter(lambda f: f.startswith(
    'crime-incident-reports-'),files))
csv_files

['crime-incident-reports-2015.csv',
 'crime-incident-reports-2016.csv',
 'crime-incident-reports-2017.csv',
 'crime-incident-reports-2018.csv',
 'crime-incident-reports-2019.csv',
 'crime-incident-reports-2020.csv',
 'crime-incident-reports-2022.csv']

In [ ]:
df_final = pd.DataFrame()
for file in csv_files:
  df = pd.read_csv(file)
  df_final=pd.concat(objs=[df_final,df]).reset_index(drop = True)



<ipython-input-17-1f06f561c24b>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
<ipython-input-17-1f06f561c24b>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [ ]:
df_final.shape

(525981, 17)

In [ ]:
df_final.isnull().sum()

INCIDENT_NUMBER             0
OFFENSE_CODE                0
OFFENSE_CODE_GROUP     172728
OFFENSE_DESCRIPTION         0
DISTRICT                 3188
REPORTING_AREA              0
SHOOTING               351798
OCCURRED_ON_DATE            0
YEAR                        0
MONTH                       0
DAY_OF_WEEK                 0
HOUR                        0
UCR_PART               172825
STREET                  11208
Lat                     22530
Long                    22530
Location                    0
dtype: int64

In [ ]:
#Replace empty locations with null values
df_final['REPORTING_AREA'] = np.where(df_final['REPORTING_AREA'].str.strip() == '',
                                      np.nan,
                                      df_final['REPORTING_AREA'])

df_final['OFFENSE_CODE_GROUP'] = df_final['OFFENSE_CODE_GROUP'].str.upper() # converting offense code group and offense description to uppercase letters
df_final['OFFENSE_DESCRIPTION'] = df_final['OFFENSE_DESCRIPTION'].str.upper()

#Converting the location column to a set to sepearte latitude and longitude into seperate columns
df_final['LOCATION'] = df_final['Location'].apply(lambda x : eval(x))
df_final['LATITUDE'] = df_final['LOCATION'].apply(lambda x : x[0])
df_final['LONGITUDE'] = df_final['LOCATION'].apply(lambda x : x[1])

#Checking for the incorrect locations and adding null values for the invalid locations
df_final['LOCATION'] = np.where((abs(df_final['LATITUDE'])<=10)|(abs(df_final['LONGITUDE'])<=10) ,
                                      np.nan,
                                      df_final['LOCATION'])
df_final['LATITUDE'] = np.where((abs(df_final['LATITUDE'])<=10)|(abs(df_final['LONGITUDE'])<=10) ,
                                      np.nan,
                                      df_final['LATITUDE'])
df_final['LONGITUDE'] = np.where((abs(df_final['LATITUDE'])<=10)|(abs(df_final['LONGITUDE'])<=10) ,
                                      np.nan,
                                      df_final['LONGITUDE'])

#Converting Null in shooting to 0 and the 'Y' to 1
df_final['SHOOTING'] = np.where(
    df_final['SHOOTING'].isna(),  # condition for values that are NaN
    0,                            # value to set for NaN values
    np.where(
        df_final['SHOOTING'].isin(['Y', 1]), # condition for values that are 'Y' or 1
        1,                                   # value to set for 'Y' or 1
        0                                    # value to set for all other values
    )
)


In [ ]:
df_final = df_final.drop(columns = ['Location','Lat','Long']) # Dropping unnecessary columns

In [ ]:
#Removing duplicates of reporting area and district based on combinations of reporting area , district and locations
reporting_area_df = df_final[['REPORTING_AREA',
                              'DISTRICT',
                              'LOCATION']].drop_duplicates()[['REPORTING_AREA',
                                                              'LOCATION']].drop_duplicates(ignore_index = 
                                                                                           True)
district_df = df_final[['REPORTING_AREA',
                              'DISTRICT',
                              'LOCATION']].drop_duplicates()[['DISTRICT',
                                                              'LOCATION']].drop_duplicates(ignore_index = 
                                                                                           True)
#Take a random location from reporting area
reporting_area_df = reporting_area_df.groupby(['REPORTING_AREA'])['LOCATION'].apply(
    lambda x: x.sample(1)).reset_index(level = 
                                       0).reset_index(drop =
                                                      True)
#Take a random location from district
district_df = district_df.groupby(['DISTRICT'])['LOCATION'].apply(
    lambda x: x.sample(1)).reset_index(level = 
                                       0).reset_index(drop =
                                                      True)

In [ ]:
#Renaming the columns
district_df.columns = ['DISTRICT','DISTRICT_LOCATION']
reporting_area_df.columns = ['REPORTING_AREA','REPORTING_AREA_LOCATION']

In [ ]:
#Merging with district and reporting area locations
df_final = pd.merge(df_final,district_df,how = 'left', on = ['DISTRICT'])
df_final = pd.merge(df_final,reporting_area_df,how = 'left', on = ['REPORTING_AREA'])

In [ ]:
df_final.columns

Index(['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'UCR_PART',
       'STREET', 'LOCATION', 'LATITUDE', 'LONGITUDE', 'DISTRICT_LOCATION',
       'REPORTING_AREA_LOCATION'],
      dtype='object')

In [ ]:
#Replacing null values with district and reporting area locations
df_final['LOCATION'] = np.where(df_final['LOCATION'].isnull(),
                                      np.where(df_final['REPORTING_AREA_LOCATION'].isnull(),
                                              df_final['DISTRICT_LOCATION'],
                                              df_final['REPORTING_AREA_LOCATION']),
                                      df_final['LOCATION'])

#Removing all null location values
df_final = df_final[df_final['LOCATION'].isnull() == False].reset_index(drop = True)
#Dropping unneccessary columns 
df_final = df_final.drop(columns = ['REPORTING_AREA_LOCATION',
                                    'DISTRICT_LOCATION',
                                    'LATITUDE',
                                   'LONGITUDE'])
#Re-run code on the modified location to get the correct lat and long info
df_final['LATITUDE'] = df_final['LOCATION'].apply(lambda x : x[0])
df_final['LONGITUDE'] = df_final['LOCATION'].apply(lambda x : x[1])

#Obtaining the correct offense code description
offense_types = pd.read_excel('rmsoffensecodes.xlsx', engine = 'openpyxl')
offense_types.columns = ['OFFENSE_CODE','OFFENSE_DESCRIPTION_MOD']
offense_types = offense_types.drop_duplicates(subset = ['OFFENSE_CODE'],ignore_index = True)

#To get correct offense descriptions
df_final = pd.merge(df_final,offense_types,how = 'left',on = 'OFFENSE_CODE')
df_final['OFFENSE_DESCRIPTION_MOD'] = np.where(df_final['OFFENSE_DESCRIPTION_MOD'].isna(),
         df_final['OFFENSE_DESCRIPTION'],
         df_final['OFFENSE_DESCRIPTION_MOD'])

#Take all offense codes and groups where code group is not null
offense_df = df_final[df_final['OFFENSE_CODE_GROUP'].isnull()== False][[
    'OFFENSE_CODE','OFFENSE_CODE_GROUP','OFFENSE_DESCRIPTION_MOD']]
offense_df = offense_df.drop_duplicates().reset_index(drop = True)
offense_df = offense_df.drop(columns = ['OFFENSE_DESCRIPTION_MOD'])
offense_df.columns = ['OFFENSE_CODE','OFFENSE_CODE_GROUP_MOD']


#Joining with the offenses df to get the code groups
df_final = pd.merge_asof(df_final.sort_values(by = 'OFFENSE_CODE').reset_index(drop = True),
                    offense_df.sort_values(by = 'OFFENSE_CODE').reset_index(drop = True),
                    on = 'OFFENSE_CODE'
                   )


#Making minor offense code group changes
df_final['OFFENSE_CODE_GROUP_MOD'] = np.where(df_final['OFFENSE_CODE'].isin([100,99999]),
                                              np.where(df_final['OFFENSE_CODE'] == 100,
                                                       'HOMICIDE',
                                                       'OTHER'),
                                              df_final['OFFENSE_CODE_GROUP_MOD'])

#Making column name modifications
df_final = df_final.drop(columns = ['OFFENSE_CODE_GROUP','OFFENSE_DESCRIPTION'])
df_final = df_final.rename( columns = {'OFFENSE_DESCRIPTION_MOD': 'OFFENSE_DESCRIPTION',
                 'OFFENSE_CODE_GROUP_MOD' : 'OFFENSE_CODE_GROUP'})
df_final = df_final.drop_duplicates().reset_index(drop = True)


In [ ]:
df_final

,INCIDENT_NUMBER,OFFENSE_CODE,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,LOCATION,LATITUDE,LONGITUDE,OFFENSE_DESCRIPTION,OFFENSE_CODE_GROUP
0,192074613,100,C11,289,1,2019-09-16 23:46:00,2019,9,Monday,23,NaN,WILLIAM T MORRISSEY BLVD,"(42.3248774992221, -71.1031643522542)",42.324877,-71.103164,MIGRATED REPORT - CRIMINAL HOMICIDE,HOMICIDE
1,192070453,100,A7,NaN,0,2019-09-03 23:25:00,2019,9,Tuesday,23,NaN,W EAGLE ST,"(42.38135785377728, -71.03768162799254)",42.381358,-71.037682,MIGRATED REPORT - CRIMINAL HOMICIDE,HOMICIDE
2,192006868,100,B3,NaN,1,2019-01-26 17:10:00,2019,1,Saturday,17,NaN,WITHINGTON ST,"(42.3353844395738, -71.1067796487276)",42.335384,-71.106780,MIGRATED REPORT - CRIMINAL HOMICIDE,HOMICIDE
3,192040692,100,C11,NaN,0,2019-05-30 22:47:00,2019,5,Thursday,22,NaN,TREMLETT ST,"(42.2876490873579, -71.0406527576907)",42.287649,-71.040653,MIGRATED REPORT - CRIMINAL HOMICIDE,HOMICIDE
4,192002129,100,B2,317,1,2019-01-08 16:31:00,2019,1,Tuesday,16,NaN,CRAWFORD ST,"(42.3113097533378, -71.08610019760567)",42.311310,-71.086100,MIGRATED REPORT - CRIMINAL HOMICIDE,HOMICIDE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523978,I172105779,3831,B3,442,0,2017-12-22 20:30:00,2017,12,Friday,20,Part Three,W PARK ST,"(42.29427188, -71.07887359)",42.294272,-71.078874,M/V - LEAVING SCENE - PROPERTY DAMAGE,MOTOR VEHICLE ACCIDENT RESPONSE
523979,I172105783,3831,B2,262,0,2017-12-22 14:30:00,2017,12,Friday,14,Part Three,MORELAND ST,"(42.32209079, -71.07477877)",42.322091,-71.074779,M/V - LEAVING SCENE - PROPERTY DAMAGE,MOTOR VEHICLE ACCIDENT RESPONSE
523980,I172105760,3831,B3,428,0,2017-12-22 16:30:00,2017,12,Friday,16,Part Three,CLARKWOOD ST,"(42.28013735, -71.09079826)",42.280137,-71.090798,M/V - LEAVING SCENE - PROPERTY DAMAGE,MOTOR VEHICLE ACCIDENT RESPONSE
523981,I182096577,3831,B3,430,0,2018-11-30 13:54:00,2018,11,Friday,13,Part Three,BLUE HILL AVE,"(42.28480303, -71.09114391)",42.284803,-71.091144,M/V - LEAVING SCENE - PROPERTY DAMAGE,MOTOR VEHICLE ACCIDENT RESPONSE


In [ ]:
#Create a geolocator class object
geolocator = Nominatim(user_agent = "geoapiProject",timeout=1000000)
location_address = df_final[['LOCATION']].drop_duplicates().reset_index(drop = True)
geolocator.reverse('42.34003269410177, -71.08923607501121').raw['address']

{'amenity': 'Northeastern University',
 'house_number': '360',
 'road': 'Huntington Avenue',
 'suburb': 'Fenway / Kenmore',
 'city': 'Boston',
 'county': 'Suffolk County',
 'state': 'Massachusetts',
 'ISO3166-2-lvl4': 'US-MA',
 'postcode': '02115',
 'country': 'United States',
 'country_code': 'us'}

In [ ]:
location_adress_list = []
cnt = 0
for i in location_address['LOCATION'].unique():
     cnt += 1
     location_adress_list.append([i[0],i[1],
                                 geolocator.reverse(str(i[0])+ 
                                                                    "," + 
                                                                    str(i[1])).raw['address']])
     

In [ ]:
address_boston = pd.DataFrame(data = location_adress_list,columns = ['LATITUDE','LONGITUDE','ADDRESS'])

In [ ]:
#Get address details 
address_boston['ROAD'] = address_boston['ADDRESS'].apply(lambda x : x.get('road'))
address_boston['NEIGHBOURHOOD'] = address_boston['ADDRESS'].apply(lambda x : x.get('neighbourhood'))
address_boston['SUBURB'] = address_boston['ADDRESS'].apply(lambda x : x.get('suburb'))
address_boston['CITY'] = address_boston['ADDRESS'].apply(lambda x : x.get('city'))
address_boston['COUNTY'] = address_boston['ADDRESS'].apply(lambda x : x.get('county'))

In [ ]:
df_final = df_final.drop(columns = ['DISTRICT','UCR_PART','STREET','REPORTING_AREA'])
df_final = pd.merge(df_final,
                    address_boston,
                    how = 'inner',
                    on = ['LATITUDE','LONGITUDE'])

In [ ]:
df_final.to_csv('crime_reports_cleaned.csv',index = False)

df_final.head()

,INCIDENT_NUMBER,OFFENSE_CODE,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,LOCATION,LATITUDE,LONGITUDE,OFFENSE_DESCRIPTION,OFFENSE_CODE_GROUP,ADDRESS,ROAD,NEIGHBOURHOOD,SUBURB,CITY,COUNTY
0,192074613,100,1,2019-09-16 23:46:00,2019,9,Monday,23,"(42.334529988231175, -71.08950599859743)",42.33453,-71.089506,MIGRATED REPORT - CRIMINAL HOMICIDE,HOMICIDE,"{'road': 'Ruggles Street', 'suburb': 'South En...",Ruggles Street,None,South End,Boston,Suffolk County
1,I182101462,301,0,2018-12-16 17:30:00,2018,12,Sunday,17,"(42.334529988231175, -71.08950599859743)",42.33453,-71.089506,ROBBERY - STREET,ROBBERY,"{'road': 'Ruggles Street', 'suburb': 'South En...",Ruggles Street,None,South End,Boston,Suffolk County
2,192076520,400,0,2019-09-22 21:50:00,2019,9,Sunday,21,"(42.334529988231175, -71.08950599859743)",42.33453,-71.089506,MIGRATED REPORT - AGGRAVATED ASSAULT/AGGRAVATE...,ROBBERY,"{'road': 'Ruggles Street', 'suburb': 'South En...",Ruggles Street,None,South End,Boston,Suffolk County
3,192066401,413,0,2019-08-22 16:00:00,2019,8,Thursday,16,"(42.334529988231175, -71.08950599859743)",42.33453,-71.089506,ASSAULT - AGGRAVATED - BATTERY,AGGRAVATED ASSAULT,"{'road': 'Ruggles Street', 'suburb': 'South En...",Ruggles Street,None,South End,Boston,Suffolk County
4,192037802,423,0,2019-05-20 15:30:00,2019,5,Monday,15,"(42.334529988231175, -71.08950599859743)",42.33453,-71.089506,ASSAULT - AGGRAVATED,AGGRAVATED ASSAULT,"{'road': 'Ruggles Street', 'suburb': 'South En...",Ruggles Street,None,South End,Boston,Suffolk County
